In [1]:
import pandas as pd
import os

df = pd.read_csv("wbc.csv")


In [2]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [3]:
# Encabezado del dataframe
df.head() 

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
id                         569 non-null int64
diagnosis                  569 non-null object
radius_mean                569 non-null float64
texture_mean               569 non-null float64
perimeter_mean             569 non-null float64
area_mean                  569 non-null float64
smoothness_mean            569 non-null float64
compactness_mean           569 non-null float64
concavity_mean             569 non-null float64
concave points_mean        569 non-null float64
symmetry_mean              569 non-null float64
fractal_dimension_mean     569 non-null float64
radius_se                  569 non-null float64
texture_se                 569 non-null float64
perimeter_se               569 non-null float64
area_se                    569 non-null float64
smoothness_se              569 non-null float64
compactness_se             569 non-null float64
concavity_se               569 non

In [5]:
# Categories relacionadas con la columna diagnosis - puede ser un Atributo Categorico
df["diagnosis"].value_counts()


#evaluo si luego lo ajusto con stratified ya que hay diferencia entre estas dos categorias
# sin embargo solo es de clasificacion, no de regresion, por lo que no se si deba hacerlo

B    357
M    212
Name: diagnosis, dtype: int64

In [6]:
# Como la data siempre va a ser la misma (no cambia ni se incrementa o disminuye) usamos la funcion
# train_test_split para usar seed o random state garantizando que siempre genere los mismos indices

from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size = 0.3, random_state = 42)

print(len(train_set), "Train Set data +", len(test_set), "Test Set data")

398 Train Set data + 171 Test Set data


In [7]:
#print(train_set)
# Busco valor puntual para validar resultados
print(df.loc[df['radius_mean'] == 12.470])

         id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
204   87930         B        12.47         18.60           81.09      481.9   
497  914580         B        12.47         17.31           80.45      480.1   

     smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
204          0.09965            0.1058         0.08005              0.03821   
497          0.08928            0.0763         0.03609              0.02369   

        ...       texture_worst  perimeter_worst  area_worst  \
204     ...               24.64            96.05       677.9   
497     ...               24.34            92.82       607.3   

     smoothness_worst  compactness_worst  concavity_worst  \
204            0.1426             0.2378           0.2671   
497            0.1276             0.2506           0.2028   

     concave points_worst  symmetry_worst  fractal_dimension_worst  \
204                0.1015          0.3014                  0.08750   
497    

In [8]:
#print(test_set)

In [9]:
# Defino la funcion Euclidiana para determinar la distancia entre dos puntos (x,y);a(x1,y1) y b(x2,y2)
# a y b son tipo listas
# a: punto o coordenada de los datos Test o datos de entrada a clasificar
# b: punto o coordenada de los datos Train
# Calculo la distancia entre el punto a y cada uno de los puntos o coordenadas del Train (b)
# Identifico la distancia mas corta de todas, la minima, 
# para ubicarlo al mas cercano, al de la minima distancia y asi saber a que clase (diagnosis) 
# pertenece.


def dist_euclidiana(a,b): 
    componentes_distancia = list()
    err = 0
    mensaje_err = ''
    lena = len(a)
    
    # Validacion para determinar que hayan llegado las listas o valores de cada punto de igual dimension
    
    if not len(a) == len(b):
        err = 1
        mensaje_err = 'Vectores con dimension diferente. Validar base de datos'
        distancia = 0
    else: # la dimension de las listas a y b son iguales
        for i in range(lena):
            #print(lena)
            componentes_distancia.append((a[i]-b[i])**2)
            distancia = sum(componentes_distancia)**0.5
            #print(distancia)
    return distancia, err, mensaje_err
            
 # ver try y except   
#dist_euclidiana([0,0],[2,2])

            

In [17]:
def sort_by_distance(list_of_distances):
    return list_of_distances[0]

In [28]:
# Recorro cada punto o coordenada del Test_set y calculo su distancia a cada punto del Train_set
# debo determinar la len de cada dataframe test y train

def knn_estimator(train_set=train_set, test_set=test_set, k=3):
    
    #la = len(test_set)
    #lb = len(train_set)
    clases_resultantes = []
    #print(la, lb)
    for test_index, fila_test in test_set.iterrows():# Iterrows deja iterar las filas de un dataframe. Devuelve índice y valor.
        
        distancias = [] 
        test_radius_mean, test_concave_mean = fila_test[["radius_mean", "concave points_mean"]]
    #for fila_test in range(la): # 0 - 0 ... 171
        #for fila_train in range(lb):# 0 - 1 - 2 ... 398
        for train_index, fila_train in train_set.iterrows():
            # estoy parado en fila cero de test y fila cero de train
            # debo capturar ambas coordenadas
            
            #print(type(test_radius_mean))
            #test_radius_mean = test_set.iloc[fila_test, 2]
            #test_concave_mean = test_set.iloc[fila_test, 9]
            train_radius_mean, train_concave_mean = fila_train[["radius_mean", "concave points_mean"]]
            #train_radius_mean = train_set.iloc[fila_train, 2]
            #train_concave_mean = train_set.iloc[fila_train, 9]
            #print("{}-{}-{}-{}".format(test_radius_mean, test_concave_mean, train_radius_mean, train_concave_mean))
            distancia = dist_euclidiana([test_radius_mean,test_concave_mean],[train_radius_mean,train_concave_mean])[0]
            #print(distancia)
            distancias.append((distancia, test_index, train_index))
            # Ordeno distancias
            
        distancias.sort(key=sort_by_distance)
        tuplas_K_menores_distancias = distancias[:k]
        
        #print([tupla[2] for tupla in tuplas_K_menores_distancias]) #Indices de menores distancias en df
        #print(tuplas_K_menores_distancias)
        
        diagnosticos = df.iloc[[tupla[2] for tupla in tuplas_K_menores_distancias] , 1]
        conteo_clases = diagnosticos.value_counts()
        
        clase = conteo_clases.index[0]
        
        clases_resultantes.append((test_index, clase))
        #print (diagnosticos)
    print(clases_resultantes)
    
    return clases_resultantes
        
                    
knn_estimator()
  

[(204, 'B'), (70, 'M'), (131, 'M'), (431, 'B'), (540, 'B'), (567, 'M'), (369, 'M'), (29, 'M'), (81, 'B'), (477, 'B'), (457, 'B'), (167, 'M'), (165, 'B'), (329, 'M'), (527, 'B'), (83, 'M'), (511, 'B'), (556, 'B'), (101, 'B'), (535, 'M'), (73, 'B'), (394, 'B'), (393, 'M'), (425, 'B'), (305, 'B'), (76, 'B'), (384, 'B'), (555, 'B'), (362, 'B'), (72, 'M'), (551, 'B'), (158, 'B'), (424, 'B'), (532, 'B'), (222, 'B'), (55, 'B'), (10, 'M'), (281, 'B'), (6, 'M'), (90, 'B'), (104, 'B'), (353, 'M'), (422, 'B'), (211, 'B'), (275, 'B'), (109, 'B'), (520, 'B'), (557, 'B'), (531, 'B'), (284, 'B'), (264, 'M'), (30, 'M'), (208, 'B'), (528, 'M'), (145, 'B'), (464, 'B'), (320, 'B'), (82, 'M'), (39, 'B'), (271, 'B'), (79, 'B'), (2, 'M'), (564, 'M'), (462, 'B'), (334, 'B'), (228, 'B'), (118, 'M'), (78, 'M'), (188, 'B'), (331, 'B'), (196, 'B'), (11, 'M'), (395, 'B'), (177, 'M'), (538, 'B'), (482, 'B'), (235, 'B'), (255, 'B'), (144, 'B'), (380, 'B'), (132, 'B'), (333, 'B'), (86, 'B'), (250, 'M'), (274, 'M'), 

[(204, 'B'),
 (70, 'M'),
 (131, 'M'),
 (431, 'B'),
 (540, 'B'),
 (567, 'M'),
 (369, 'M'),
 (29, 'M'),
 (81, 'B'),
 (477, 'B'),
 (457, 'B'),
 (167, 'M'),
 (165, 'B'),
 (329, 'M'),
 (527, 'B'),
 (83, 'M'),
 (511, 'B'),
 (556, 'B'),
 (101, 'B'),
 (535, 'M'),
 (73, 'B'),
 (394, 'B'),
 (393, 'M'),
 (425, 'B'),
 (305, 'B'),
 (76, 'B'),
 (384, 'B'),
 (555, 'B'),
 (362, 'B'),
 (72, 'M'),
 (551, 'B'),
 (158, 'B'),
 (424, 'B'),
 (532, 'B'),
 (222, 'B'),
 (55, 'B'),
 (10, 'M'),
 (281, 'B'),
 (6, 'M'),
 (90, 'B'),
 (104, 'B'),
 (353, 'M'),
 (422, 'B'),
 (211, 'B'),
 (275, 'B'),
 (109, 'B'),
 (520, 'B'),
 (557, 'B'),
 (531, 'B'),
 (284, 'B'),
 (264, 'M'),
 (30, 'M'),
 (208, 'B'),
 (528, 'M'),
 (145, 'B'),
 (464, 'B'),
 (320, 'B'),
 (82, 'M'),
 (39, 'B'),
 (271, 'B'),
 (79, 'B'),
 (2, 'M'),
 (564, 'M'),
 (462, 'B'),
 (334, 'B'),
 (228, 'B'),
 (118, 'M'),
 (78, 'M'),
 (188, 'B'),
 (331, 'B'),
 (196, 'B'),
 (11, 'M'),
 (395, 'B'),
 (177, 'M'),
 (538, 'B'),
 (482, 'B'),
 (235, 'B'),
 (255, 'B'),
 (144,

In [31]:
# Busco dato puntual. Index, columna
df.iloc[210, 1]

'M'

In [11]:
# calcular la moda. En KNN se hace referencia a la moda de los datos
    maxi = max(map(vecinos.count, vecinos))
    return


IndentationError: unexpected indent (<ipython-input-11-9f1916dc90b0>, line 2)

In [15]:
dd = pd.DataFrame({'A': [0, 1, 2], 'B': [3, 4, 5], 'C': [6, 7, 8]})

for index, row in dd.iterrows():
    a, c = row[['A', 'C']]
    print("{}-{}".format(a, c))

0-6
1-7
2-8
